# 부스팅
부스팅은 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법이다.  
앞의 모델을 보완해 나가면서 일련의 예측기를 학습시키는 것임.  

---
## 에이다부스팅
**Adaptive boosting** . 가장 인기있는 부스팅 방법.  
> 1. 알고리즘의 기반이 되는 첫 번째 분류기를 학습 데이터셋에 학습 후 예측  
2. 알고리즘이 잘못 분류한 샘플의 가중치를 상대적으로 높임.  
3. 업데이트된 가중치를 사용하여 학습 데이터셋에 다시 학습 후 예측.  
4. 위 과정 반복  
  
> 모든 예측기가 학습을 마치면 배깅이나 페이스팅과 비슷한 방식으로 최종 예측을 만듦.  
그런데 이 방법은 각 예측기가 이전 예측기가 학습하고 평가한 후에야 학습할 수 있기 때문에 **병렬화를 할 수 없음**  
따라서 배깅이나 페이스팅보다 확장성은 떨어짐

In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5
)

**n_estimators**와 **learning_rate**는 트레이드오프관계라고 함.(왜..??)  
**algorithm** 은 기본적으로 저걸 쓰는 듯

---
## 그래디언트 부스팅
**Gradient boosting** . 에이다부스트처럼 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가하는 것.  
그래디언트부스팅은 샘플의 가중치를 수정하는 대신에 **이전 예측기가 만든 잔여오차에 새로운 예측기를 학습함**  

결정 트리를 기반 예측기로 사용하는 그래디언트 부스팅 방법이 많이 쓰임  
이를 **그래디언트 트리 부스팅**, 또는 **그래디언티드 부스티드 회귀 트리(GBRT)** 라고 함.

In [2]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)

**GradientBoostingRegressor** 클래스로 GBRT를 구현할 수 있음. 랜덤포레스트처럼 트리의 성장을 제어하는 매개변수를 갖고 있음.  
**learning_rate** 매개변수는 각 트리의 기여 정도를 조절함. 0.1처럼 낮게 설정하면 앙상블예측기는 학습데이터를 학습하기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아짐. (아, 그래서 서로 트레이드오프 관계라는 거)  
> learning_rate크고 n_estimator작을수록 : 모델 과소적합?  
learning_rate작고 n_estimator클수록 : 모델 과대적합

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

x, y = make_moons(n_samples=500, noise=0.3, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [5]:
gbrt.fit(x_train, y_train)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [8]:
for pred in gbrt.staged_predict(x_test):
    print(pred.shape)

(125,)
(125,)
(125,)


**staged_predict** 메서드로 각 예측기의 예측값을 순서대로 출력할 수도 있음.  
가장 오차가 적은 예측기가 몇 번째인지를 판단하는 것으로 **최적의 트리 수** 를 추정할 수도 있음.

---
## 확률적 그래디언트 부스팅
GradientBoostingRegressor에서 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있음.  
**subsample** 매개변수를 이용하면 됨. 예를 들어 subsample=0.25일 경우 각 트리는 무작위로 선택된 25%의 학습 데이터로 학습함.  
역시 편향이 높아지는 대신 분산이 낮아짐.  

최적화된 그래디언트 부스팅 구현으로 **XGBoost 라이브러리**가 유명함  
매우 빠른 속도, 확장성, 이식성이 장점임.

In [10]:
import xgboost

xgb_reg = xgboost.XGBRegressor()

In [11]:
xgb_reg.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
from sklearn.metrics import mean_squared_error
y_pred = xgb_reg.predict(x_test)
mean_squared_error(y_test, y_pred)

0.08984928005708914